## 기본 langchain UnstructuredPowerPointLoader 사용


In [1]:
from langchain_community.document_loaders import UnstructuredPowerPointLoader
from dotenv import load_dotenv
from pathlib import Path
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
import os
import sys

from agent.config import VECTORSTORE_DIR, PROPOSAL_DIR

load_dotenv()

# Chroma 벡터스토어 생성
def get_vectorstore():
    collection_name = os.getenv('BASIC_PROPOSAL_DB_NAME')
    vectorstore = Chroma(
        persist_directory=VECTORSTORE_DIR['db'],
        embedding_function=OllamaEmbeddings(model="nomic-embed-text"),
        collection_name=collection_name
    )
    return vectorstore

# 벡터 스토어에 파일이 존재하는지 확인
def check_file_in_vectorstore(db, file_name):
    result = db.get()

    # 메타데이터가 있는지 확인
    if not result['metadatas']:
        return False
    
    found = False
    for metadata in result['metadatas']:
        if 'source' in metadata and file_name in metadata['source']:
            found = True
            break
    # if found:
    #     print(f"{file_name} 파일이 이미 존재합니다.")
    # else:
    #     print(f"{file_name} 파일이 존재하지 않습니다.")
        
    return found

# 메타데이터에서 리스트 형태의 메타데이터를 추출
def check_list_in_metadata(metadata):
    list_fields = []
    
    for key, value in metadata.items():
        if isinstance(value, list):
            list_fields.append(key)
            # print(f"{key} 필드는 리스트 형태입니다.")
            
    return list_fields

# PPT 파일을 로드하여 vectorstore에 저장
def add_ppt_to_vectorstore(db, ppt_file_path):
    loader = UnstructuredPowerPointLoader(ppt_file_path)
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    documents = loader.load_and_split(text_splitter=splitter)
    
    for doc in documents:
        list_fields = check_list_in_metadata(doc.metadata)
        if list_fields:
            for field in list_fields:
                if doc.metadata[field]:
                    doc.metadata[field] = ','.join(doc.metadata[field])
    print(documents[0])
    db.add_documents(documents)

# vectorstore 디렉토리가 존재하는지 확인하고 없으면 생성
def ensure_vectorstore_directory():
    persist_dir = Path(VECTORSTORE_DIR['db'])
    if not persist_dir.exists():
        persist_dir.mkdir(parents=True)
        print(f"벡터스토어 디렉토리 생성됨: {persist_dir}")
    else:
        print(f"벡터스토어 디렉토리가 이미 존재함: {persist_dir}")

# PPT 파일을 찾아서 목록 생성
def get_source_pptx_files():
    # documents 디렉토리 경로
    doc_dir = Path(PROPOSAL_DIR['source'])
    
    # .pptx 파일만 찾아서 목록 생성
    pptx_files = [f for f in doc_dir.glob('*.pptx') if not f.name.startswith('~$')]
    
    if not pptx_files:
        print("PPT 파일이 존재하지 않습니다.")
        return []
    
    # print(f"발견된 PPT 파일 목록:")
    # for file in pptx_files:
    #     print(f"- {file.name}")
    
    return pptx_files

In [2]:
ensure_vectorstore_directory()
db = get_vectorstore()
# print(db.get())

try:
    pptx_files = get_source_pptx_files()
except Exception as e:
    print(f"PPT 파일을 찾는 중 오류 발생: {e}")


if not pptx_files:
    sys.exit(0)
else:
    for pptx in pptx_files:
        if check_file_in_vectorstore(db, pptx.name):
            print(f"{pptx.name} 파일이 이미 존재합니다.")
        else:
            try:    
                pptx_path = os.path.join(PROPOSAL_DIR['source'], pptx.name)    
                print(f"{pptx_path} 파일을 벡터스토어에 저장합니다.")
                add_ppt_to_vectorstore(db, pptx_path)
            except Exception as e:
                print(f"PPT 파일을 벡터스토어에 저장하는 중 오류 발생: {e}")



벡터스토어 디렉토리가 이미 존재함: /Users/andrew/Projects/proposal-agent/data/vectorstore
/Users/andrew/Projects/proposal-agent/data/source/제안발표_샘플.pptx 파일을 벡터스토어에 저장합니다.


/var/folders/3w/r_xtz2nd5jd9bwqk5b3mm14h0000gn/T/ipykernel_10870/496152576.py:19: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_function=OllamaEmbeddings(model="nomic-embed-text"),


page_content='2018. 06



제안 개요

사업추진 방안

사업관리 방안

맺음말

목  차



제안 개요

사업추진 방안

사업관리 방안

맺음말

제안의 목적

제안범위

사업추진전략



제안의 목적

Ⅰ. 제안 개요

목  표

통합건강증진

개인별 관리 변경

개인정보파기

관리기능 구축

정신보건사업 

업무지침 현행화

DW 통계 기능

개선

보안 취약점 

분석/평가 컨설팅

 보안취약점에 대한

보호 대책 수립

성공적인 지역보건의료정보시스템 기능개발 및 보안컨설팅 수행

지역보건의료정보시스템 기능개발 및 보안컨설팅 필요

법령 및 업무지침 변경에 

따른 프로그램 개선 필요

통합건강증진 효율적

관리운영 및 일원화 필요

지역보건의료정보시스템의 

정보보호체계 강화

수요자와 사용자 중심의 서비스 운영지원 및 정보 보호체계 강화

정신보건사업의 업무지침 변경에 따른 프로그램 기능 개선 필요

지역보건법 개정에 따른 개인정보 파기관리 기능 구축 필요

지역사회 통합건강증진사업 개인별 관리기능 강화 필요

건강증진 서비스 제공 및 정책수립 지원을 위한 통계분석체계 구축 필요

 전자적 침해 행위에 대응하기 위한

  지역보건의료정보시스템의 관리적, 

  기술적, 물리적 정보보호 체계 강화 

  필요



제안범위

Ⅰ. 제안 개요

개인별 건강증진 관리 기능 개발

통합건강증진사업 대상자 관리기능 추가                         

문답지 관리를 통한 대상자 건강정보관리 기능   

개인별 통합 서비스 관리 기능 개선 

정신보건사업 업무지침 현행화

ISP 계획 수립 관리 기능 구축                         

자살위험군관리 기능 구축 

자살위험군관리 구축 및 업무 지침 변경에 따른 현행화

신체활동사업 시스템 구축

보건사업 영역에 신체활동 시스템 및 주요 실적관리 페이지 8개 신설

   (사업관리, 대상자이력조회, 교육프로그램, 교육명단관리, 개인관리서비스, 개인명단관리, 홍보캠

In [3]:
db  = get_vectorstore()

retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 10,
        "fetch_k": 10,
        "lambda_mult": 0.6
    }
)

results = retriever.invoke("no code AI 관련 제안서를 작성하기 위한 목차를 정리해줘")

for doc in results:
    # print(doc.metadata)
    print(doc.page_content)



구축방안

^ 로그인 정보 없음

5

계정등록

로그인

ID/PW로그인

ID/PW 확인

사용자 계정 조회

아이디, 이름, 생년월일, 성별을 입력하여 조회할 수 있는 기능 구현

계정조회

인증서로그인

인증서 및 

계정 확인

권한별 업무화면

2

4

1

3

인증서 로그인

등록 인증서로 로그인하는 기능 구현

인증서등록

^ 등록된 계정 조회

^ 등록된 인증서 없음

인증서 등록관리

GPKI, NPKI를 사용자 개인 인증서를 지역보건의료정보시스템 계정에 매핑하는 기능 과 삭제 기능 구현

사용자 계정 정보를 아이디, 이름, 생년월일, 성별을 입력하여 조회할 수 있는 기능 구현

사용자의 인증서 GPKI, NPKI 등을 이용하여 지역보건의료정보시스템 계정에 매핑하는 기능 또는 등록된 인증서를 삭제하는 기능 구현 

로그인 시 인증서 로그인 시 인증서를 선택하고 인증하고 등록된 인증서의 사용자 계정 정보로 로그인 할 수 기능 구현

인증서 로그인으로 로그인 시 해당 인증서로 등록된 사용자 계정이 다중인(2개 이상) 경우 로그인 할 사용자 계정을 선택 할 수 있는 화면(상위보건기관명, 해당보건기관명, 아이디, 사용자명 목록) 구현

인증서 로그인 시 선택한 인증서가 등록되지 않은 경우 등록된 인증서가 없다는 메시지 정보와  인증서 등록 화면 생성

공인인증서 로그인 관리 기능 개발

인증서 확인

등록된 인증서 다중인 경우 선택한 계정으로 로그인 되는 기능 구현

등록된 인증서가 없는 경우는 메시지 처리와 인증서 등록화면 호출



1

2

3

4

1

3

2

4

기타 지원기능 개발

Ⅱ. 사업추진 방안

한의약통계

 기능개발

한의약건강증진사업 운영현황

구축방안

한의약건강증진 운영현황

한의과 진료실/한의약 건강증진 프로그램 : 전년도, 금년도 입력 처리

고용형태

일반직, 임기제, 공중보건의, 무기계약, 기간제, 기타 항목을 숫자만 입력

공공보건 한의약업무 수행인력

면허 및 자격
1

보안 취약점 분석·평가 및 보호대

In [23]:
SAMPLE_PPTX = "data/source/제안발표_샘플.pptx"

loader = UnstructuredPowerPointLoader(SAMPLE_PPTX)
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = loader.load_and_split(text_splitter=splitter)

for doc in documents:
    print(doc.metadata)
    print(doc.page_content)

{'source': 'data/source/제안발표_샘플.pptx'}
2018. 06



제안 개요

사업추진 방안

사업관리 방안

맺음말

목  차



제안 개요

사업추진 방안

사업관리 방안

맺음말

제안의 목적

제안범위

사업추진전략



제안의 목적

Ⅰ. 제안 개요

목  표

통합건강증진

개인별 관리 변경

개인정보파기

관리기능 구축

정신보건사업 

업무지침 현행화

DW 통계 기능

개선

보안 취약점 

분석/평가 컨설팅

 보안취약점에 대한

보호 대책 수립

성공적인 지역보건의료정보시스템 기능개발 및 보안컨설팅 수행

지역보건의료정보시스템 기능개발 및 보안컨설팅 필요

법령 및 업무지침 변경에 

따른 프로그램 개선 필요

통합건강증진 효율적

관리운영 및 일원화 필요

지역보건의료정보시스템의 

정보보호체계 강화

수요자와 사용자 중심의 서비스 운영지원 및 정보 보호체계 강화

정신보건사업의 업무지침 변경에 따른 프로그램 기능 개선 필요

지역보건법 개정에 따른 개인정보 파기관리 기능 구축 필요

지역사회 통합건강증진사업 개인별 관리기능 강화 필요

건강증진 서비스 제공 및 정책수립 지원을 위한 통계분석체계 구축 필요

 전자적 침해 행위에 대응하기 위한

  지역보건의료정보시스템의 관리적, 

  기술적, 물리적 정보보호 체계 강화 

  필요



제안범위

Ⅰ. 제안 개요

개인별 건강증진 관리 기능 개발

통합건강증진사업 대상자 관리기능 추가                         

문답지 관리를 통한 대상자 건강정보관리 기능   

개인별 통합 서비스 관리 기능 개선 

정신보건사업 업무지침 현행화

ISP 계획 수립 관리 기능 구축                         

자살위험군관리 기능 구축 

자살위험군관리 구축 및 업무 지침 변경에 따른 현행화

신체활동사업 시스템 구축

보건사업 영역에 신체활동 시스템 및 주요 실적관리 페이지 8개 신설

   (사업관리, 대상자이력조회, 교육프로그램, 교육명